# Coursework 2
(core codebase to be imported as modules)

In [71]:
import numpy as np

from perceptron import Perceptron, KernelPerceptron
from kernels import polynomial_kernel, radial_basis_kernel
from utilities import train_test_split, data_split, y_encode

In [2]:
# Loading Data
data = np.loadtxt("dtrain123.dat")
y = data[:, 0]
X = data[:, 1:]

In [3]:
# Build test XOR dataset
a = np.array([[1,1], [-1,-1], [-1,1], [1,-1]])
b = np.array([1,1,-1,-1])

In [4]:
# Run for regular unkernalised perceptron
p = Perceptron()
p.train(a, b)
p.predict(a)

array([ 0.,  0.,  0.,  0.])

In [5]:
# Run for 2-order polynomial kernel
p_2 = KernelPerceptron(polynomial_kernel, 2)
p_2.train(a,b)
p_2.predict_proba(a)

array([ 4.,  4., -4., -4.])

In [6]:
p_2.gram

array([[4, 4, 0, 0],
       [4, 4, 0, 0],
       [0, 0, 4, 4],
       [0, 0, 4, 4]])

## Test for differentiating between 1s and 3s

In [7]:
idx = y!=2
X_z = X[idx,:]
y_z = y[idx]
y_z = (y_z == 1).astype(int) - (y_z == 3).astype(int)
n_1 = sum(y==1)
n_3 = sum(y==3)
print(n_1, n_3)

120 70


### Regular perceptron

In [8]:
p = Perceptron()
p.train(X_z, y_z)

In [10]:
p.predict(X_z).dot(y_z)/len(y_z)

1.0

In [11]:
p.predict(X_z).dot(y_z)

190.0

In [12]:
X_z.shape

(190, 256)

In [13]:
### Polynomial Kernel
for d in range(1,5):
    p_k = KernelPerceptron(kernel=polynomial_kernel, k_params=d)
    p_k.train(X_z, y_z)
    error = p_k.predict(X_z).dot(y_z)/len(y_z)
    print("{} : {}".format(d, error))

1 : 1.0
2 : 1.0
3 : 1.0
4 : 1.0


## DT's code

from utilities import preprocessing, train_test_split,y_encode,data_split

In [14]:
data = np.loadtxt("zipcombo.dat")

In [21]:
data.shape

(9298, 257)

This will be a one vs all scenario.

In [79]:
%%time
# first class is zero

# matrix to hold predictions


X,y = data_split(data,y_col=0)
X_train,X_test,y_train,y_test = train_test_split(X,y,seed=3)

predictions = np.zeros((X_test.shape[0],10))


y_train_i = y_encode(y_train,0)

P = KernelPerceptron(polynomial_kernel, k_params=2) # quadratic kernel
P.train(X_train,y_train_i)
print(P.M)
predictions[:,0] = P.predict_proba(X_test)

[ 113.]
CPU times: user 4.26 s, sys: 455 ms, total: 4.71 s
Wall time: 3.51 s


In [80]:
%time
for i in range(1,10):
    y_train_i = y_encode(y_train,i)
    print(y_train_i[0:10])
    P = KernelPerceptron(polynomial_kernel, k_params=2) # quadratic kernel
    print(P.M)
    P.train(X_train,y_train_i)
    print(P.M)
    predictions[:,i] = P.predict_proba(X_test)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.2 µs
[-1 -1 -1 -1 -1 -1  1 -1  1 -1]
0
[ 67.]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[ 170.]
[-1 -1 -1 -1 -1 -1 -1  1 -1 -1]
0
[ 169.]
[-1 -1 -1  1 -1 -1 -1 -1 -1 -1]
0
[ 195.]
[-1 -1  1 -1 -1 -1 -1 -1 -1 -1]
0
[ 177.]
[-1 -1 -1 -1 -1  1 -1 -1 -1 -1]
0
[ 132.]
[-1  1 -1 -1 -1 -1 -1 -1 -1  1]
0
[ 126.]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
0
[ 212.]
[-1 -1 -1 -1  1 -1 -1 -1 -1 -1]
0
[ 219.]


In [81]:
preds = np.argmax(predictions,axis=1)

In [83]:
preds.shape

(3100,)

In [86]:
predictions[1:10, :]

array([[-60320.02714077,  27614.18052106, -82780.94313469, -41779.40487842,
        -45979.4198174 , -35621.94251204, -49939.80421103, -84824.6280945 ,
        -75699.25437149, -41698.6615646 ],
       [-62447.91388161,  52886.4057866 , -74525.1036188 , -61937.64626742,
        -51193.9452106 , -47064.60382842, -33251.43309396, -72275.32760312,
        -99674.5636128 , -46691.68444866],
       [-22893.1468252 , -64782.40568071, -45348.23520323, -37275.72406586,
        -13366.19013453, -31805.80254775, -37368.8313287 , -56356.68295584,
        -38933.1474884 , -24069.54631964],
       [-33891.43956923, -74985.00855432, -63511.32107652, -48670.54023377,
        -68615.3780766 , -45103.31862861, -57337.48343712, -93939.62380212,
         19459.51674215, -14095.78600147],
       [-43086.08813015, -20536.33397904, -40535.56199513, -49262.90595434,
        -61034.71709711, -37599.02131695, -14660.46173554,  10683.14560147,
        -61247.08656483, -19996.65974356],
       [-15948.38206199, 

In [87]:
# Success 
(preds == y_test).mean()

0.94290322580645158

In [91]:
preds[1:30]

array([1, 1, 4, 8, 7, 6, 4, 8, 5, 9, 1, 4, 7, 0, 1, 0, 8, 1, 2, 2, 6, 6, 0,
       6, 3, 0, 0, 8, 8])

In [92]:
y_test[1:20]

array([ 1.,  1.,  4.,  8.,  7.,  6.,  4.,  8.,  5.,  9.,  1.,  4.,  7.,
        0.,  1.,  0.,  8.,  1.,  2.])

In [89]:
preds == y_test

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)